<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/Treinamento_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Configurações Iniciais 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
from numpy import loadtxt
import string, re
import itertools
import nltk
import plotly.offline as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.callbacks import EarlyStopping
py.init_notebook_mode(connected=True)
%matplotlib inline
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

## Lendo a Base

In [3]:
# Base1 Com StopWords
PATH_BASE1 = "/content/drive/MyDrive/TCC/dados/processadas/Base1_classificada.csv"

# Base2 Sem StopWords
PATH_BASE2 = "/content/drive/MyDrive/TCC/dados/processadas/Base2_classificada.csv"

# Base1 Com StopWords
PATH_BASE1_JUNTO_COM_DA_LEILA_BALANCEADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base1JuntaHateENotHateDaLeila_balanceada.csv"
#MATRIZ_CBOW_300_BASE_1 = loadtxt("/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base1JuntaHateENotHateDaLeila_balanceada_CBOW300.CSV", delimiter=',')

# Base2 Sem StopWords
PATH_BASE2_LEILA_LIMPISSIMA_BALANCEADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_maior4_menor25_limpissima_balanceada.csv"
#MATRIZ_CBOW_300_BASE_2_LEILA = loadtxt("/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base2_maior4_menor25_limpissima_balanceada_CBOW300.CSV", delimiter=',')

PATH_BASE_2_CLASSIFICADA = r"/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_classificada_balanceada.csv" #entrada
MATRIZ_CBOW_300_BASE_2 = loadtxt("/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base2_classificada_balanceada_CBOW300.csv", delimiter=',') #saida

In [4]:
MATRIZ_CBOW_300_BASE_2.shape

(19044, 300)

In [5]:
#trainDF = pd.read_csv(PATH_BASE2_LEILA_LIMPISSIMA_BALANCEADA, index_col=0)
trainDF = pd.read_csv(PATH_BASE_2_CLASSIFICADA, index_col=0)

In [6]:
# Agrupa dados por label
trainDF.groupby('label').count()

,text
label,
0,5287
1,5287


In [27]:
# Pega a média de caracteres dos tweets de toda a base
average_len = int(trainDF['text'].apply(lambda x: len(str(x).split(' '))).mean())
average_len

9

In [28]:
# Pega a média de caracteres dos tweets de toda a base
max_len = int(trainDF['text'].apply(lambda x: len(str(x).split(' '))).max())
max_len

55

In [29]:
palavras_unicas = list ( trainDF['text'].str.split(' ', expand=True).stack().unique() )
palavras_unicas = set(map(lambda x: re.findall(r'^\D+', x)[0] if re.findall(r'^\D+', x) != [] else "a" ,palavras_unicas)) 
quantidade_palavras_unicas = len( palavras_unicas )
quantidade_palavras_unicas

19043

In [30]:
trainDF['label'].astype(int)
print(type(trainDF['label'].astype(int)[0]))
print(type(trainDF['label'][0]))

<class 'numpy.int64'>
<class 'numpy.int64'>


#TF-IDF (NÃO CONSEGUIMOS USAR NO MODELO CNN)

In [31]:
#word level tf-idf
#td-idf gera matriz com 9140 colunas (4 palavras estão sendo perdidas ao usar o regex \w{1,})
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features= quantidade_palavras_unicas)
X = tfidf_vect.fit_transform(trainDF['text'])

print(X)
print(type(X))
X = X.toarray()
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X, trainDF['label'], test_size=0.3, random_state=2)
# Não conseguimos usar o c-bow300

  (0, 16964)	0.3765471882742689
  (0, 18328)	0.20751828460116187
  (0, 1860)	0.3173859901260883
  (0, 16262)	0.1986877040719613
  (0, 14738)	0.24915740231151903
  (0, 8733)	0.36059867282262115
  (0, 16317)	0.2290646490984455
  (0, 7613)	0.1781697155942826
  (0, 1856)	0.6058438963282801
  (0, 15657)	0.17766542818847925
  (1, 7639)	0.4272447175136453
  (1, 9277)	0.26382653808116807
  (1, 7416)	0.33883211941225166
  (1, 12458)	0.4272447175136453
  (1, 11641)	0.39494157190968027
  (1, 1597)	0.3365583227044106
  (1, 9357)	0.39494157190968027
  (1, 10316)	0.1590064789418787
  (2, 7051)	0.13056046661910514
  (2, 12307)	0.13556789501336003
  (2, 14047)	0.13056046661910514
  (2, 13157)	0.14789007266539347
  (2, 2746)	0.35725167664865803
  (2, 8888)	0.45607844945015413
  (2, 6706)	0.18895658145884775
  :	:
  (10569, 13019)	0.32098192457661046
  (10569, 9041)	0.36690339939859073
  (10570, 6234)	0.44472573693475087
  (10570, 18986)	0.44472573693475087
  (10570, 6346)	0.41252505333527023
  (10570, 

In [32]:
# X.shape
# print(X)

In [33]:
#create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

train_x_text, valid_x_text, train_y_text, valid_y_text = model_selection.train_test_split(trainDF['text'], trainDF['label'], test_size=0.3, random_state=2)

In [34]:
# len(word_index) + 1

In [35]:
# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x_text), maxlen=max_len)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x_text), maxlen=max_len)

In [36]:
# print(X.shape[1])


# CNN model

In [37]:
# def create_cnn(input_size):
#     # Add an Input Layer
#     input_layer = layers.Input((input_size,1,))

#     middle_layer = layers.Dropout(0.3)(input_layer)
    
#     # middle_layer = layers.Convolution1D(100, 3, activation="relu")(middle_layer)
#     # middle_layer = layers.Convolution1D(100, 3, activation="relu")(middle_layer)
#     # middle_layer = layers.Dropout(0.3)(middle_layer)
#     # middle_layer = layers.MaxPooling1D()(middle_layer)
    
#     middle_layer = layers.Convolution1D(100, 3, activation="relu")(middle_layer)
#     middle_layer = layers.Convolution1D(100, 3, activation="relu")(middle_layer)
#     middle_layer = layers.Dropout(0.3)(middle_layer)
#     middle_layer = layers.MaxPooling1D()(middle_layer)
    
#     middle_layer = layers.Flatten()(middle_layer)
#     output_layer = layers.Dense(64, activation="relu")(middle_layer)

#     # Compile the model
#     model = models.Model(inputs=input_layer, outputs=output_layer)
#     model.compile(optimizer='adam', loss='mean_squared_error', metrics = ['acc', 'mae'])
  
#     return model

In [38]:
# def create_cnn(input_size):
#     # Add an Input Layer
#     input_layer = layers.Input((input_size,1,))

#     input_layer = layers.SpatialDropout1D(0.3)(input_layer)

#     # Add the word embedding Layer
#        # embedding_layer = layers.Embedding(X.shape[1] + 1, 300, weights=[MATRIZ_CBOW_300_BASE_2], trainable=True)(input_layer)
#     embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

#     # Add the convolutional Layer
#     #conv_layer = layers.Convolution1D(100, 3, activation="relu")(input_layer)
#     conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

#     # Add the pooling Layer
#     pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

#     # Add the output Layers
#     output_layer1 = layers.Dense(64, activation="relu")(pooling_layer)
#     output_layer1 = layers.Dropout(0.25)(output_layer1)
#     output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

#     # Compile the model
#     model = models.Model(inputs=input_layer, outputs=output_layer2)
#     model.compile(optimizer='adam', loss='mean_squared_error', metrics = ['acc', 'mae'])
  
#     return model

In [39]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, validation, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label, epochs=15) #adicionamos epoca
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    return metrics.accuracy_score(predictions, validation)

In [58]:
embedding_layer = layers.Embedding(19044,
                           300,
                            # weights=[MATRIZ_CBOW_300_BASE_2],
                            input_length=max_len,
                            trainable=True)

In [65]:
def create_cnn(input_size):
    sequence_input = layers.Input(shape=(input_size,), dtype='float32')
    embedded_sequences = embedding_layer(sequence_input)
    x = layers.Conv1D(32, 5, activation='relu',name="COVID")(embedded_sequences)
    x = layers.MaxPooling1D(5)(x)
    x = layers.Conv1D(32, 5, activation='relu')(x)
    x = layers.MaxPooling1D(5)(x)
    x = layers.Conv1D(32, 5, activation='relu')(x)
    x = layers.MaxPooling1D(25)(x)  # global max pooling
    x = layers.Flatten()(x)
    x = layers.Dense(32, activation='relu')(x)
    preds = layers.Dense(2, activation='softmax')(x)

    model = models.Model(inputs=sequence_input, outputs=preds)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='rmsprop',
                 metrics = ['acc', 'mae'])
    

    # #  Compile the model
    # model = models.Model(inputs=sequence_input, outputs=preds)
    # model.compile(optimizer='adam', loss='mean_squared_error', metrics = ['acc', 'mae'])
  
    return model

In [63]:
classifier = create_cnn(max_len)

accuracyCNN = train_model(classifier,train_seq_x, train_y_text, valid_seq_x, valid_y_text, is_neural_net=True)

print ("CNN, Word Embeddings - accuracy",  accuracyCNN)

ValueError: ignored

In [66]:
classifier = create_cnn(19043)

accuracyCNN = train_model(classifier,train_x, train_y, valid_x, valid_y, is_neural_net=True)

print ("CNN, Word Embeddings - accuracy",  accuracyCNN)

Epoch 1/15
232/232 [==============================] - 85s 353ms/step - loss: 0.6939 - acc: 0.4961 - mae: 0.5000
Epoch 2/15
232/232 [==============================] - 81s 348ms/step - loss: 0.6932 - acc: 0.5007 - mae: 0.5000
Epoch 3/15
232/232 [==============================] - 81s 347ms/step - loss: 0.6932 - acc: 0.4991 - mae: 0.5000
Epoch 4/15
232/232 [==============================] - 81s 347ms/step - loss: 0.6932 - acc: 0.4867 - mae: 0.5000
Epoch 5/15
232/232 [==============================] - 81s 347ms/step - loss: 0.6932 - acc: 0.4918 - mae: 0.5000
Epoch 6/15
232/232 [==============================] - 81s 348ms/step - loss: 0.6932 - acc: 0.4932 - mae: 0.5000
Epoch 7/15
232/232 [==============================] - 81s 347ms/step - loss: 0.6932 - acc: 0.5001 - mae: 0.5000
Epoch 8/15
232/232 [==============================] - 81s 347ms/step - loss: 0.6932 - acc: 0.4959 - mae: 0.5000
Epoch 9/15
 93/232 [===========>..................] - ETA: 48s - loss: 0.6932 - acc: 0.4966 - mae: 0.500

KeyboardInterrupt: ignored

https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/



https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html



Fazer testes com as Bases BALANCEADAS